In [1]:
import PyNomad
import numpy as np
import matplotlib.pyplot as plt



### Objective function class

In [2]:
class ObjectiveFunction_min: # Class for objective synthetic functions
    def __init__(
        self, benchmark, dim=2, size=None):
        self.dim = dim
        # self.negate = negate

        if benchmark == "Ackley":
            self.f = self.ackley_function

            if size is None:
                self.size = 64
            else:
                self.size = size
            self.upper_bound = 32
            self.lower_bound = -32
            
        elif benchmark == "Hartmann":
            try:
                assert dim == 6
            except AssertionError:
                raise ValueError("Hartmann function is only defined for dim=6.")

            self.f = self.hartmann_6d_function

            self.size = 5
            self.upper_bound = 1
            self.lower_bound = 0

        elif benchmark == "Michalewicz":
            self.f = self.michalewicz_function

            self.upper_bound = np.pi
            self.lower_bound = 0
            if self.dim == 2:
                self.size = 64

            elif self.dim == 4:
                self.size = 10

            else:
                raise ValueError("Choose dim=2 or dim=4 for Michalewicz function.")
        else:
            raise ValueError(
                "Choose a valid benchmark function in ['Ackley', 'Hartmann', 'Michalewicz']."
            )

    # Ackley Function
    def ackley_function(self, x):
        a = 20
        b = 0.2
        c = 2 * np.pi
        d = len(x)
        sum1 = np.sum(x**2)
        sum2 = np.sum(np.cos(c * x))
        return -a * np.exp(-b * np.sqrt(sum1 / d)) - np.exp(sum2 / d) + a + np.e
    
    def michalewicz_function(self, x):
        d = len(x) # 10
        return -np.sum(np.sin(x) * np.sin(np.arange(1, self.dim + 1) * x**2 / np.pi)**(2 * d))
    
    def hartmann_6d_function(self, x):
        """Computes the 6-dimensional Hartmann function"""
        # Constants for the Hartmann 6D function
        alpha = np.array([1.0, 1.2, 3.0, 3.2])
        A = np.array([
            [10, 3, 17, 3.50, 1.7, 8],
            [0.05, 10, 17, 0.1, 8, 14],
            [3, 3.5, 1.7, 10, 17, 8],
            [17, 8, 0.05, 10, 0.1, 14]
        ])
        P = np.array([
            [1312, 1696, 5569, 124, 8283, 5886],
            [2329, 4135, 8307, 3736, 1004, 9991],
            [2348, 1451, 3522, 2883, 3047, 6650],
            [4047, 8828, 8732, 5743, 1091, 381]
        ])*1e-4
        
        
        x = np.asarray(x)  # Ensure the input is a numpy array
        # x.reshape((1,6))
        result = 0.0
        # for i in range(4):  # Loop through the 4 components
        #     sum_exp = 0.0
        #     for j in range(6):  # Loop through the 6 dimensions
        #         sum_exp += A[i][j] * (x[j] - P[i][j])**2
        #     print("sum_exp", sum_exp, "\n")
        #     print("alpha[i] ", alpha[i], "\n")
        #     result -= alpha[i] * np.exp(-sum_exp)
        #     print("result ", result, "\n")
        # # return result

        outer = 0
        for ii in range(4):
            inner = 0
            for jj in range(6):
                xj = x[jj]
                Aij = A[ii, jj]
                Pij = P[ii, jj]
                inner = inner + Aij*(xj-Pij)**2

            new = alpha[ii] * np.exp(-inner)
            outer = outer + new

        result = -outer
        return result
        
    # def bb_Ackley(self, y):
    #     return obj.ackley_function(x)
        
    # Generate Input Space
    def create_input_space(self):
        X = np.linspace(self.lower_bound, self.upper_bound, self.size)
        X = np.array([X for _ in range(self.dim)])
        X = np.meshgrid(*X)
        ch2xy = np.array([x.flatten() for x in X]).T
        return ch2xy

    # Generate Normalized True Response
    def generate_true_response(self, input_space):
        response = np.array([self.f(x) for x in input_space])
        response = (response - response.min()) / (response.max() - response.min())
        return response



### Ackley

In [3]:
if __name__ == "__main__":
    # Initialize the Objective Function
    obj = ObjectiveFunction_min("Ackley", dim=2)
    # obj = ObjectiveFunction_min("Michalewicz", dim=2)
    # obj = ObjectiveFunction_min("Michalewicz", dim=4)
    # obj = ObjectiveFunction_min("Hartmann", dim=6)

    # Generate Input Space and True Responses
    ch2xy = obj.create_input_space()
    response = obj.generate_true_response(ch2xy)
    
    print("Input space shape:", ch2xy.shape)
    print("Response shape:", response.shape)
    
    def bb_Ackley(y):
        # print("y: ", y, "\n")
        # print("y: ", y.get_coord(0), "\n")
        # print("y.get_coord(0): ", int(y.get_coord(0)), "\n")
        
        # x = ch2xy[y[0]]
        # return response[y[0]]
        f = response[int(y.get_coord(0))]
        # y.set_bb_output(0,f)
        y.setBBO(str(f).encode("UTF-8"))
        return 1 # 1: success

    seed = 42
    np.random.seed(seed)

    # CMA-ES will optimize indices within the range [0, len(ch2xy)-1]
    index_bounds = [0, len(ch2xy) - 1]
    y0 = [int(len(ch2xy) // 2)]  # Start at the middle index
    lb = [index_bounds[0]]
    ub = [index_bounds[1]]
    
    params = ['BB_OUTPUT_TYPE OBJ','MAX_BB_EVAL 150', 'SEED 42', "DISPLAY_ALL_EVAL yes", "DISPLAY_STATS BBE OBJ", 'SOLUTION_FILE C:\\Users\\nicle\\anaconda3\envs\\boo\\pyNomad\\Ackley_results.txt'
              , 'STATS_FILE C:\\Users\\nicle\\anaconda3\envs\\boo\\pyNomad\\Ackley_stats.txt']
    
    result = PyNomad.optimize(bb_Ackley,y0,lb,ub,params)
    # print("output: ", output, "\n")
    fmt = ["{} = {}".format(n,v) for (n,v) in result.items()]
    output = "\n".join(fmt)
    print("\nNOMAD results \n" + output + " \n")

    # [ x_return , f_return , h_return, nb_evals , nb_iters ,  stopflag ] = PyNomad.optimize(bb,y0,lb,ub,params)
    # print ('\n NOMAD outputs \n X_sol={} \n F_sol={} \n H_sol={} \n NB_evals={} \n NB_iters={} \n'.format(x_return,f_return,h_return,nb_evals,nb_iters))



Input space shape: (4096, 2)
Response shape: (4096,)

NOMAD results 
x_best = [1503.0]
f_best = 0.7035782600442829
h_best = 0.0
nb_evals = 118
nb_iters = 0
run_flag = 1
stop_reason = No termination (all). Mesh minimum precision reached (Algo) 

